<a href="https://colab.research.google.com/github/edipasq/bigdata-challenge/blob/main/Wirelessproducts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-25 16:37:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-01-25 16:37:12 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
#url="https://<bucket name>.s3.amazonaws.com/user_data.csv"
url="https://bucket2-etl-class.s3.us-west-1.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show(5)

+---+----------+---------+-----------+-------------------+--------------+---------+
| id|first_name|last_name|active_user|     street_address|         state| username|
+---+----------+---------+-----------+-------------------+--------------+---------+
|  1|    Cletus|  Lithcow|      false|78309 Riverside Way|      Virginia|ibearham0|
|  2|       Caz|   Felgat|      false|83 Hazelcrest Place|       Alabama| wwaller1|
|  3|     Kerri|  Crowson|      false|     112 Eliot Pass|North Carolina|ichesnut2|
|  4|   Freddie|    Caghy|      false|    15 Merchant Way|      New York|  tsnarr3|
|  5|   Sadella|    Deuss|      false|   079 Acker Avenue|     Tennessee|fwherrit4|
+---+----------+---------+-----------+-------------------+--------------+---------+
only showing top 5 rows



In [6]:
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
w_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
w_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [7]:
from pyspark.sql.functions import col

print(df.count())

In [9]:
print(w_df.count())

9002021


In [10]:
w_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [11]:
print(w_df.count())
w_df = w_df.dropna()
print(w_df.count())
df = w_df.dropDuplicates()
print(w_df.count())

9002021
9001051
9001051


In [8]:
w_df2 = w_df.select(["review_id", "customer_id", "product_id", "product_parent",'review_date'])
w_df2.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import *

d=w_df2.select('review_id','customer_id','product_id','product_parent',date_format(col('review_date'),"yyyy-MM-dd").alias('rev').cast("date"))

In [10]:
d.show()

+--------------+-----------+----------+--------------+----------+
|     review_id|customer_id|product_id|product_parent|       rev|
+--------------+-----------+----------+--------------+----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543|2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650|2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154|2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704|2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300|2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488|2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554|2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702|2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711|2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876|2015-08-31|
|R3MRWNNR8CBTB7|    1972249|B00U4NATNQ|     577878727|2015-08-31|
|R1DS6DKTUXAQK3|   10956619|B00SZEFDH8|     654620704|2015-08-31|
| RWJM5E0T

In [15]:
d.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- rev: date (nullable = true)



In [11]:
d = d.withColumnRenamed("rev", "review_date")

In [73]:
d.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702| 2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876| 2015-08-31|
|R3MRWNNR8CBTB7|    1972249|B00U4NATNQ|     577878727| 2015-08-31|
|R1DS6DKTUXAQK3|   10956619|B00SZEFDH8|     654620704| 2015-08

In [12]:
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cz0jfhlrydbo.us-west-1.rds.amazonaws.com:5432/bigdatadb"
config = {"user":"postgres",
          "password": "*************",
          "driver":"org.postgresql.Driver"}

In [18]:
d.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
p_df2 = w_df.select(["product_id", "product_title"])
p_df2.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YL0EKWE|LG G4 Case Hard T...|
|B00XK95RPQ|Selfie Stick Fibl...|
|B00SXRXUKO|Tribe AB40 Water ...|
|B009V5X1CE|RAVPower® Element...|
|B00D93OVF0|Fosmon Micro USB ...|
+----------+--------------------+
only showing top 5 rows



In [20]:
p_df22 = p_df2.dropDuplicates()

In [21]:
p_df22.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
cust=w_df.select('customer_id')

In [23]:
print(cust.count())
cust = cust.dropna()
print(cust.count())


9001051
9001051


In [24]:
cust_df = cust.groupBy("customer_id").count()

In [25]:
print(cust_df.count())

5197356


In [26]:
cust_df.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   46909180|    6|
|   42560427|    7|
|   43789873|    3|
|   22037526|    2|
|   34220092|    2|
+-----------+-----+
only showing top 5 rows



In [27]:
df_index = cust_df.withColumnRenamed("count", "customer_count")

In [28]:
df_index.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [29]:
df_index.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
#df_index = df_index.filter(col("customer_id")  == 10230)
#df_index.sort(col("customer_id")).show(truncate=False)

In [30]:
df_index.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [13]:
vines=w_df.select('review_id','star_rating','helpful_votes','total_votes','vine')

In [14]:
vines.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
|R11LOHEDYJALTN|          5|            0|          0|   N|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|
|R2L15IS24CX0LI|          5|            0|          0|   N|
|R1DJ8976WPWVZU|          3|            0|          0|   N|
|R3MRWNNR8CBTB7|          5|            0|          0|   N|
|R1DS6DKTUXAQK3|          5|            0|          0|   N|
| RWJM5E0TWUJD2|          5|            0|          0|   N|
|R1XTJKDYNCRGAC|          1|            

In [15]:
vines.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [16]:
vines = vines.withColumn("star_rating",col("star_rating").cast("integer"))

In [17]:
vines.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [18]:
vines.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)